In [9]:
import requests
import pandas as pd
import numpy as np
from datetime import datetime, timedelta
from time import time, sleep

In [2]:
def transform_df(file_name):
    df = pd.read_json(file_name)
    df.drop('disclaimer', axis = 1, inplace = True)
    df.drop('time', axis = 1, inplace = True)
    df.drop(['updated', 'updatedISO'], inplace = True)
    diff = df.diff().rename(columns = {'bpi': 'Diff'}).Diff.values
    df['diff'] = diff
    df.fillna(0, inplace = True)
    df['label'] = np.where(df['diff'] > 0, True, False)
    df.drop('diff', axis = 1, inplace= True)
    return df

In [3]:
# s = sched.scheduler(time.time, time.sleep)
def run_api():
    yesterday = (datetime.now() - timedelta(1)).strftime('%Y-%m-%d')
    !curl 'https://api.coindesk.com/v1/bpi/historical/close.json?start=2011-12-31&end={yesterday}' > current_price.json
    full_df = transform_df('current_price.json')
    full_df.to_json('full_data.json')
#     s.enter(86400, 1)

In [4]:
run_api()

  % Total    % Received % Xferd  Average Speed   Time    Time     Time  Current
                                 Dload  Upload   Total   Spent    Left  Speed
100 73139  100 73139    0     0   655k      0 --:--:-- --:--:-- --:--:--  655k


In [10]:
while True:
    sleep(86400 - time() % 86400)
    run_api()

KeyboardInterrupt: 